#### This notebook is text only supervised classification using Fasttext

In [1]:
# Fasttext main branch has issue in calculation of F1 score. hence using this label from github
!pip install git+https://github.com/facebookresearch/fastText.git@b64e359d5485dda4b4b5074494155d18e25c8d13 -q

In [2]:
from prepare_ft_file import *
import pandas as pd
from sklearn.model_selection import train_test_split
import fasttext

In [3]:
# Load MP3 mapped data
data = pd.read_json('song_lyric_map.json')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data['Lyric'], data['Genre'], random_state=42, test_size=0.3)

In [5]:
write_ft_file(X_train, y_train, 'traindata.txt')
write_ft_file(X_test, y_test, 'testdata.txt')

In [17]:
# Auto tune Fast Text Model
ft_auto_tune_model = fasttext.train_supervised(input="traindata.txt", 
                                    autotuneValidationFile='testdata.txt', 
                                    autotuneModelSize="1M", 
                                    autotuneMetric='f1',
                                    dim=50)

In [18]:
test_score = ft_auto_tune_model.test('testdata.txt')
train_score = ft_auto_tune_model.test('traindata.txt')
print('Training')
print(train_score)
print('Validation')
print(test_score)

Training
(863, 1.0, 1.0)
Validation
(370, 0.8837837837837837, 0.8837837837837837)


In [19]:
ft_auto_tune_model.save_model('ft_autotuned_model_song_lyrics.bin')